In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV

import import_ipynb
# Импортируем полностью модули для передачи глобальных переменных
from utils import make_datasets
import models

In [2]:
def hybrid_recommendation(user_id, book_id, df_ratings, df_books, df_tags, df_book_tags, weights=None):
    """Функция объединяет прогнозы разных моделей и выводит список рекомендуемых книг.
    :param user_id: идентификатор пользователя
    :param book_id: идентификатор книги
    :param df_ratings: DataFrame с рейтингами
    :param df_books: DataFrame с информацией о книгах
    :param df_tags: DataFrame с тегами
    :param df_book_tags: DataFrame с привязанными тегами к книгам
    :param weights: весовые коэффициенты для моделей [популярность, похожесть, коллаборативная фильтрация]

    :return: список рекомендованных книг"""
    
    # Определим веса для каждой из рекомендаций
    if weights is None:
        weights = [2, 3, 4]
    
    # Получаем рекомендации книг по каждой модели в соответствии с весами
    popularity_rec = models.get_popularity_recommendation_ids(df_ratings, weights[0])
    content_rec = models.get_similar_books_ids(book_id, df_book_tags, df_tags, df_books, df_ratings, weights[1])
    collaborative_rec = models.get_recommendations_svd(user_id, df_ratings, weights[2])
    
    # Применяем весовые коэффициенты
    recommendations = (popularity_rec + content_rec + collaborative_rec)
    
    # Убираем дублирующиеся элементы и возвращаем уникальный список
    unique_recs = list(set(recommendations))
    
    return unique_recs

# Запуск

In [5]:
# Определяем запуск из-под скрипта:
if __name__ == '__main__':
    data_path = os.path.abspath('../data')
    book_id = 6621
    user_id = 315

    # Загружаем даныне
    print('Загружаем данные...')
    df_ratings, df_books, df_tags, df_book_tags, df_users = make_datasets(data_path)

    # Создаем матрицу взаимодействий
    print('Создаем матрицу взаимодействий...')
    # df_interaction_matrix = generate_user_book_similarity_matrix(df_ratings, df_books, df_book_tags, df_tags)
    
    # Обучаем модель SVD
    print('Обучаем модель SVD...')
    # models.best_params = prepare_model_svd_mode(df_ratings, 'increment')
    models.best_params = {'n_factors': 100, 'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.1}

    print('Получаем гибридные рекомендации...')
    unique_recs = hybrid_recommendation(user_id, book_id, df_ratings, df_books, df_tags, df_book_tags)
    print(unique_recs)

Загружаем данные...
Создаем матрицу взаимодействий...
Обучаем модель SVD...
Получаем гибридные рекомендации...
[9566, 2850, 3080, 6920, 4557, 5207, 6590]
